<a href="https://colab.research.google.com/github/ChanukaDasun/ML-APP/blob/main/Notebooks/Vehicle-Faults-Prediction-NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# data proccess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel("./TestData.xlsx")

In [ ]:
df = df.dropna(subset=['ENGINE_COOLANT_TEMP']).reset_index(drop=True)
df['TROUBLE_CODES'] = df['TROUBLE_CODES'].fillna("No error")
df.shape, df.isnull().sum(), df['TROUBLE_CODES'].unique()

((33964, 33),
 TIMESTAMP                          0
 MAKE                              55
 MODEL                             55
 CAR_YEAR                          55
 ENGINE_POWER                      55
 AUTOMATIC                         55
 VEHICLE_ID                         0
 BAROMETRIC_PRESSURE(KPA)       23761
 ENGINE_COOLANT_TEMP                0
 FUEL_LEVEL                     30981
 ENGINE_LOAD                     3600
 AMBIENT_AIR_TEMP               30348
 ENGINE_RPM                       705
 INTAKE_MANIFOLD_PRESSURE        9438
 MAF                            22153
 LONG TERM FUEL TRIM BANK 2     33336
 FUEL_TYPE                      24705
 AIR_INTAKE_TEMP                  180
 FUEL_PRESSURE                  33827
 SPEED                            471
 SHORT TERM FUEL TRIM BANK 2    33335
 SHORT TERM FUEL TRIM BANK 1     9155
 ENGINE_RUNTIME                 22088
 THROTTLE_POS                     456
 DTC_NUMBER                       254
 TROUBLE_CODES                      

In [ ]:
from copy import deepcopy

data = deepcopy(df[["ENGINE_POWER", "ENGINE_COOLANT_TEMP", "ENGINE_LOAD", "ENGINE_RPM", "INTAKE_MANIFOLD_PRESSURE",
                        "AIR_INTAKE_TEMP", "SPEED", "THROTTLE_POS", "TIMING_ADVANCE", "HOURS", "DAYS_OF_WEEK", "MONTHS"]])

unique_code = df["TROUBLE_CODES"].unique()
trouble_code_mapping_dict = {code: index for index,code in enumerate(unique_code)}
data["TROUBLE_CODES_MAPPED"] = df["TROUBLE_CODES"].map(trouble_code_mapping_dict)
data = data.interpolate(method="linear")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33964 entries, 0 to 33963
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ENGINE_POWER              33964 non-null  float64
 1   ENGINE_COOLANT_TEMP       33964 non-null  float64
 2   ENGINE_LOAD               33964 non-null  float64
 3   ENGINE_RPM                33964 non-null  float64
 4   INTAKE_MANIFOLD_PRESSURE  33964 non-null  float64
 5   AIR_INTAKE_TEMP           33964 non-null  float64
 6   SPEED                     33964 non-null  float64
 7   THROTTLE_POS              33964 non-null  float64
 8   TIMING_ADVANCE            33964 non-null  float64
 9   HOURS                     33964 non-null  float64
 10  DAYS_OF_WEEK              33964 non-null  float64
 11  MONTHS                    33964 non-null  float64
 12  TROUBLE_CODES_MAPPED      33964 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 3.4 MB


In [ ]:
import firebase_admin
from firebase_admin import db, credentials


ref = db.reference("Time_Data")
data = {"DAYS": 0, "HOURS": 1, "MONTHS": 0, "WEEKS": 0}
ref.push(data)

In [ ]:
x = [ {"a1":{1,2,3}, "a2":{1,2,3}}, {"b1":{1,2,3}, "b2":{1,2,3}} ]
len(x)

2

In [ ]:
import datetime

for i in range(5):
  date_ = datetime.date.today() + datetime.timedelta(days=i)
  print(date_)


2024-07-07
2024-07-08
2024-07-09
2024-07-10
2024-07-11


In [ ]:
import firebase_admin
from firebase_admin import db, credentials
import datetime

features = {"ENGINE_POWER", "ENGINE_COOLANT_TEMP", "ENGINE_LOAD", "ENGINE_RPM", "SPEED", "THROTTLE_POS", "AIR_INTAKE_TEMP"}

# {'a':{}, 'b':{}}
ref = db.reference("Live_obd_data")
count = 0
day = 0
obj = {}
for count in range(50, 80):
  for col in data:
    if col in features:
      obj[col] = {}
      obj[col]['VALUE'] = data[col][count]
      obj[col]['TIME'] = date_ = datetime.date.today() + datetime.timedelta(days=day)
      obj[col]['TIME'] = date_.isoformat()
  day += 1
  ref.push(obj)

print("ok")


ok


In [ ]:
data.head()

,ENGINE_POWER,ENGINE_COOLANT_TEMP,ENGINE_LOAD,ENGINE_RPM,INTAKE_MANIFOLD_PRESSURE,AIR_INTAKE_TEMP,SPEED,THROTTLE_POS,TIMING_ADVANCE,HOURS,DAYS_OF_WEEK,MONTHS,TROUBLE_CODES_MAPPED
0,1.4,80.0,0.333,1009.0,49.0,59.0,0.0,0.251,0.569,16.0,2.0,8.0,0
1,1.4,80.0,0.325,1003.0,52.0,59.0,0.0,0.251,0.565,16.0,2.0,8.0,0
2,1.4,80.0,0.329,995.0,51.0,59.0,0.0,0.251,0.573,16.0,2.0,8.0,0
3,1.4,80.0,0.325,1004.0,51.0,60.0,0.0,0.251,0.565,16.0,2.0,8.0,0
4,1.4,80.0,0.329,1005.0,49.0,60.0,0.0,0.251,0.569,16.0,2.0,8.0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

x = data.iloc[:, :-1].to_numpy().reshape(len(data), 12)
y = data.iloc[:, -1].to_numpy().reshape(len(data), 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=80)

poly = PolynomialFeatures(1, include_bias=False)

x_train_mapped = poly.fit_transform(x_train)
x_test_mapped = poly.transform(x_test)

scaler = StandardScaler()

x_train_mapped_scaled = scaler.fit_transform(x_train_mapped)
x_test_mapped_scaled = scaler.transform(x_test_mapped)

x_train_mapped_scaled.shape, x_test_mapped_scaled.shape, y_train.shape, y_test.shape

((25431, 12), (8478, 12), (25431, 1), (8478, 1))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model_1 = Sequential([
    tf.keras.Input(shape=(12,)),
    Dense(25, activation="relu"),
    Dense(18, activation="relu"),
    Dense(14, activation="linear"),
], name="model_1")

model_1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

model_1.fit(
    x_train_mapped_scaled, y_train,
    epochs=100
)

Epoch 1/100
795/795 [==============================] - 2s 2ms/step - loss: 0.5380
Epoch 2/100
795/795 [==============================] - 1s 2ms/step - loss: 0.1927
Epoch 3/100
795/795 [==============================] - 1s 2ms/step - loss: 0.1413
Epoch 4/100
795/795 [==============================] - 1s 2ms/step - loss: 0.1181
Epoch 5/100
795/795 [==============================] - 1s 2ms/step - loss: 0.1031
Epoch 6/100
795/795 [==============================] - 1s 2ms/step - loss: 0.0934
Epoch 7/100
795/795 [==============================] - 1s 2ms/step - loss: 0.0866
Epoch 8/100
795/795 [==============================] - 2s 2ms/step - loss: 0.0818
Epoch 9/100
795/795 [==============================] - 2s 2ms/step - loss: 0.0777
Epoch 10/100
795/795 [==============================] - 1s 2ms/step - loss: 0.0740
Epoch 11/100
795/795 [==============================] - 1s 2ms/step - loss: 0.0713
Epoch 12/100
795/795 [==============================] - 1s 2ms/step - loss: 0.0679
Epoch 13/100


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

z_train = model_1.predict(x_train_mapped_scaled)
sm_train = tf.nn.softmax(z_train)

z_test = model_1.predict(x_test_mapped_scaled)
sm_test = tf.nn.softmax(z_test)

loss_fn = SparseCategoricalCrossentropy()

train_loss = loss_fn(y_train, sm_train)

test_loss = loss_fn(y_test, sm_test)

train_loss, test_loss

265/265 [==============================] - 0s 1ms/step


(<tf.Tensor: shape=(), dtype=float32, numpy=0.024869494>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.074865535>)

In [ ]:
np.argmax(sm_train[1, :])

1

In [ ]:
def miss(x, y, sm):
  count = 0
  for i in range(len(x)):
    if np.argmax(sm[i]) != y[i]:
      count += 1
  return count

miss_train = miss(x_train, y_train, sm_train)
miss_test = miss(x_test, y_test, sm_test)
miss_train, miss_test

(204, 146)

In [ ]:
(miss_train/len(y_train))*100, (miss_test/len(y_test))*100

(0.8021705792143446, 1.7221042698749707)

In [ ]:
len(x_test), len(x_train)

(8478, 25431)

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

trouble_classes = ['No error','P0133', 'C0300', 'P0079P2004P3000', 'P0078U1004P3000',
        'P0079C1004P3000', 'P007EP2036P18F0', 'P007EP2036P18D0',
        'P007FP2036P18D0', 'P0079P1004P3000', 'P007EP2036P18E0',
        'P007FP2036P18E0', 'P0078B0004P3000', 'P007FP2036P18F0']

def model_predict(X):

  poly = PolynomialFeatures(1, include_bias=False)
  X_mapped = poly.fit_transform(X)
  scaler = StandardScaler()
  X_mapped_scaled = scaler.fit_transform(X_mapped)

  z = model_1.predict(X_mapped_scaled)
  sm = tf.nn.softmax(z)
  prediction_array = np.argmax(sm, axis=1) # apply for each sample(row)

  return prediction_array

In [ ]:
predict_train = model_predict(x_train)
m_count = 0
for i in range(len(y_train)):
  if predict_train[i] != y_train[i]:
    m_count+=1

m_count

795/795 [==============================] - 1s 2ms/step


204

In [ ]:
from tensorflow.keras.saving import load_model

model_1.save('my_model.keras')

In [ ]:
my_model_1 = load_model("./my_model.keras", custom_objects=None, compile=True, safe_mode=True)

trouble_classes = ['No error' ,'P0133', 'C0300', 'P0079P2004P3000', 'P0078U1004P3000',
        'P0079C1004P3000', 'P007EP2036P18F0', 'P007EP2036P18D0',
        'P007FP2036P18D0', 'P0079P1004P3000', 'P007EP2036P18E0',
        'P007FP2036P18E0', 'P0078B0004P3000', 'P007FP2036P18F0']

input_np = np.array([[1.600e+00, 7.600e+01, 2.160e-01, 1.540e+03, 6.500e+01, 3.500e+01,
        4.000e+01, 1.960e-01, 4.780e-01, 1.200e+01, 1.000e+00, 7.000e+00],
       [1.600e+00, 6.800e+01, 1.548e-01, 8.000e+02, 3.000e+01, 3.100e+01,
        0.000e+00, 2.700e-02, 5.530e-01, 1.100e+01, 1.000e+00, 8.000e+00],
       [1.600e+00, 7.900e+01, 5.060e-01, 2.063e+03, 4.600e+01, 5.400e+01,
        2.000e+01, 1.330e-01, 7.180e-01, 1.200e+01, 1.000e+00, 8.000e+00],
       [1.600e+00, 7.800e+01, 2.430e-01, 7.620e+02, 6.200e+01, 4.000e+01,
        1.600e+01, 1.100e-01, 5.180e-01, 3.000e+00, 4.000e+00, 8.000e+00],
       [1.600e+00, 7.200e+01, 4.100e-01, 1.274e+03, 6.200e+01, 3.500e+01,
        3.800e+01, 1.410e-01, 5.020e-01, 1.100e+01, 4.000e+00, 8.000e+00]])


poly = PolynomialFeatures(1, include_bias=False)
input_np_mapped = poly.fit_transform(input_np)
scaler = StandardScaler()
input_np_mapped_scaled = scaler.fit_transform(input_np_mapped)


output_np_sm = tf.nn.softmax(my_model_1.predict(input_np_mapped_scaled))

output_np = np.argmax(output_np_sm, axis=1) # apply for each sample

output_np_trouble = [trouble_classes[i] for i in output_np]

output_np_trouble

1/1 [==============================] - 0s 102ms/step


['P0133', 'P0133', 'C0300', 'C0300', 'C0300']

not at all

In [ ]:
from joblib import dump, load

dump(model_1, 'model.joblib')

['model.joblib']

In [ ]:
# load model from .joblib file
model_in = load('model.joblib')
miss(x_train_mapped_scaled, y_train, tf.nn.softmax(model_in.predict(x_train_mapped_scaled)))

224/224 [==============================] - 1s 3ms/step


36

In [ ]:
def predict_troubles(input_data, model):
  trouble_classes = ['P0133', 'C0300', 'P0079P2004P3000', 'P0078U1004P3000',
        'P0079C1004P3000', 'P007EP2036P18F0', 'P007EP2036P18D0',
        'P007FP2036P18D0', 'P0079P1004P3000', 'P007EP2036P18E0',
        'P007FP2036P18E0', 'P0078B0004P3000', 'P007FP2036P18F0']

  poly = PolynomialFeatures(1, include_bias=False)
  input_data_mapped = poly.fit_transform(input_data)
  scaler = StandardScaler()
  input_data_mapped_scaled = scaler.fit_transform(input_data_mapped)


  output_data_sm = tf.nn.softmax(my_model_1.predict(input_data_mapped_scaled))

  output_data = np.argmax(output_data_sm, axis=1) # apply for each sample

  output_troubles = [trouble_classes[i] for i in output_np]

  return output_troubles

In [ ]:
predict_troubles(input_np, my_model_1)

1/1 [==============================] - 0s 22ms/step


['P0133', 'P0133', 'C0300', 'C0300', 'C0300']

In [ ]:
def is_float(s):
  try:
    float(s)
    return True
  except:
    return False

s = "1 ,   4,  9 , 0.4,  , 1"
f = s.split(',')
a = [float(x) for x in f if is_float(x)]
a

[1.0, 4.0, 9.0, 0.4, 1.0]

In [ ]:
import numpy as np
import tensorflow as tf
def softmax(x):
  z = np.exp(x)
  s = z / z.sum(axis=0)
  return s

x = np.array([1.0, 2.0, 3.0])
print(softmax(x))
print(tf.nn.softmax(x))

[0.09003057 0.24472847 0.66524096]
tf.Tensor([0.09003057 0.24472847 0.66524096], shape=(3,), dtype=float64)


In [ ]:
model_1.save("model_1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
new_model = tf.keras.models.load_model('model_1.h5')
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                325       
                                                                 
 dense_1 (Dense)             (None, 18)                468       
                                                                 
 dense_2 (Dense)             (None, 13)                247       
                                                                 
Total params: 1040 (4.06 KB)
Trainable params: 1040 (4.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
